In [8]:
# 음성 텍스트 변환
import speech_recognition as sr
import pyaudio
import pickle

# 형태소 분석
from konlpy.tag import Okt
import pandas as pd
okt = Okt()

In [23]:
class voice:
    def __init__(self):
        self.r = sr.Recognizer()

        self.df = pd.read_csv("500_가중치.csv", encoding='utf-8')             # 전체 형태소 분석 (가중치) 파일 
        self.type_df = pd.read_csv("type_token_가중치.csv", encoding='utf-8') # 범죄 유형 분류 기준 단어 파일
        
        self.cnt = 1        # 보이스피싱 확률 변수
        self.type1_cnt = 1  # 대출사기형 확률
        self.type2_cnt = 1  # 수사기관사칭형 확률        
        self.text = ''      # 음성에서 변환된 텍스트
        self.token_dict = {} # 단어:횟수 딕셔너리 생성
        
    def call(self):
        print('\n※ 통화 시작 ※\n')
        
        while True:
            with sr.Microphone() as source:
                try:
                    voice = self.r.listen(source, phrase_time_limit=10, timeout=3)
                    self.text = self.r.recognize_google(voice, language='ko-KR')                    
                    self.ing_cnt() # 피싱 탐지 함수 호출
                    #print('▶ 통화내역 : {}'.format(self.text))

                except:
                    self.result()
                    print('\n※ 통화 종료 ※\n')
                    break
                 
    def detection(self):            
        self.token_ko = pd.DataFrame(okt.pos(self.text), columns=['단어', '형태소'])
        self.token_ko = self.token_ko[(self.token_ko['단어'].str.len() > 1)&(self.token_ko.형태소.isin(['Noun', 'Adverb']))]
            
        for i in self.token_ko.단어.values:
            if i in self.df.단어.values:
                self.cnt *= float(self.df.loc[self.df.단어==i, '확률'])
                if i not in self.token_dict:
                    self.token_dict[i] = 1
                else:
                    self.token_dict[i] = self.token_dict.get(i) + 1 
    
        if self.cnt > 100:
            self.cnt = 100  # 확률이 100%를 넘겼을 경우 100으로 초기화
            
    # 유형을 분류하는 함수 
    def categorizing(self):
        self.token_df = pd.DataFrame(zip(self.token_dict.keys(),self.token_dict.values()), columns=['의심 단어', '횟수'])
        self.token_df = self.token_df.sort_values(by='횟수', ascending=False)
        
        for i, x in zip(self.token_df['의심 단어'].values, self.token_df['횟수'].values):
            if i in self.type_df.type1_단어.values:
                self.type1_cnt *= float(self.type_df.loc[self.type_df.type1_단어==i, 'type1_확률']) ** x
            elif i in self.type_df.type2_단어.values:
                self.type2_cnt *= float(self.type_df.loc[self.type_df.type2_단어==i, 'type2_확률']) ** x
                
        if self.type1_cnt > self.type2_cnt:
            return '대출사기형'
        else:
            return '수사기관사칭형'
                        
    # 결과를 출력하는 함수
    def ing_cnt(self):
        self.detection() # 분석 함수 호출
        
        if self.cnt <=20: safe_type = '안전'
        elif self.cnt <= 40: safe_type = '의심'
        elif self.cnt <= 60: safe_type = '경고'
        else: safe_type = '위험'
            
        bolded_safe_type = "\033[1m" + safe_type + "\033[0m"
    
        print(f'▶ 보이스피싱 확률 : {self.cnt:.2f}% [{bolded_safe_type}]')
        
    def result(self):
        # 보이스피싱 확률이 의심 단계 이상일 때만 출력할 수 있도록
        if self.cnt > 20:
            self.token_csv = self.token_ko['단어'].values # csv 생성을 위한 통화음성 단어 추출 (명사, 부사)
            
            type_title = self.categorizing() # 유형 분류 함수 호출
            print(f'\n▶ 해당 음성은 {type_title} 보이스피싱일 가능성이 높습니다')
            print('▶ 보이스피싱 탐색 결과')
            display(self.token_df.head(10))

In [24]:
# 실시간 보이스피싱 탐지
v = voice()
v.call()


※ 통화 시작 ※

▶ 보이스피싱 확률 : 3.70% [안전]
▶ 보이스피싱 확률 : 6.43% [안전]
▶ 보이스피싱 확률 : 14.84% [안전]
▶ 보이스피싱 확률 : 28.97% [의심]
▶ 보이스피싱 확률 : 78.67% [위험]
▶ 보이스피싱 확률 : 95.55% [위험]

▶ 해당 음성은 대출사기형 보이스피싱일 가능성이 높습니다
▶ 보이스피싱 탐색 결과


,의심 단어,횟수
9,금융,5
0,고객,2
5,얼마,2
6,정도,2
1,저희,2
12,생각,1
18,안내,1
17,상품,1
16,모든,1
15,정보,1



※ 통화 종료 ※

